# Multi-head attention

In [22]:
import math 
import torch
from torch import nn 
from d2l import torch as d2l
from IPython.display import Image

#### Multi-head attention model implementation

In [23]:
Image(url="Multihead_attention_photo.png", width=800, height=400)

In [24]:
def transpose_qkv(X, num_heads):
	"""Transposition for parallel computation of multiple attention heads"""

	X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
	X = X.permute(0, 2, 1, 3)

	return X.reshape(-1, X.shape[2], X.shape[3])

def transpose_output(X, num_heads):
	"""Reverse the operation of transpose_qkv"""

	X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
	X = X.permute(0, 2, 1, 3)

	return X.reshape(X.shape[0], X.shape[1], -1)

In [25]:
# valid_lens: (batch_size, ) or (batch_size, no. of queries)
# Note: the mask (valid_len) used here, I copy it num_heads times

class MultiheadAttention(nn.Module):
	"""Multi-head attention"""

	def __init__(self, key_size, query_size, value_size, num_hiddens, num_heads, dropout, bias=False, **kwargs):
		super(MultiheadAttention, self).__init__(**kwargs)
		self.num_heads = num_heads
		self.attention = d2l.DotProductAttention(dropout)
		self.W_q = nn.Linear(query_size, num_hiddens, bias=bias) # (in_feature, out_feature)
		self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
		self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
		self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)
	
	def forward(self, queries, keys, values, valid_lens):
		
		# queries, keys, values:
		# (batch_size, no. of queries / keys/ values, num_hiddens) ---------------------note here we choose queries_size, value_size, key_size to be num_hiddens
		# valid_lens: (batch_size, ) or (batch_size, no. of queries)

		if valid_lens is not None:
			valid_lens = torch.repeat_interleave(valid_lens, repeats=self.num_heads, dim=0)
		
		output = self.attention(queries, keys, values, valid_lens)

		output_concat = transpose_output(output, self.num_heads)

		return self.W_o(output_concat)

The above code is written for parallel computation, avoiding loops for each head. The idea is illustrated in the flowchart below.

In [26]:
Image(url="Multihead_attention_parallel_computation_flowchart.png", width=800, height=800)

In [27]:
# test above implementation
num_hiddens, num_heads = 100, 5
attention = MultiheadAttention(key_size=num_hiddens, query_size=num_hiddens, value_size=num_hiddens, num_hiddens=num_hiddens, num_heads=num_heads, dropout=0.5, bias=False)
attention.eval()

MultiheadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [28]:
batch_size, num_queries, num_kvpairs, valid_lens = 2, 4, 6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape # final_shape should be (batch_size, num_queries, num_hiddens)

IndexError: The shape of the mask [40, 6] at index 0 does not match the shape of the indexed tensor [8, 6] at index 0